# Notebook Title

**Authorship:**
Author, *MM/DD/YYYY*
***
**Description:**
Notebook to do some cool stuff
***
**TODOs:**
 - <font color='green'> Done TODO </font>
 - <font color='orange'> WIP TODO </font>
 - <font color='red'> Queued TODO </font>
***

## Set-up

In [77]:
# The classics
import numpy as np
import pandas as pd

# Other guys
import tqdm
from pyarrow import feather

# Autoreload extension
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2

## Part 0 - Build a dummy bed file from given bim and fam

In [78]:
bim = pd.read_csv("../val/MGB.bim", sep="\t", header=None)

In [79]:
fam = pd.read_csv("../val/MGB.fam", delim_whitespace=True, header=None)

### 100 x 1000

#### PED

In [96]:
# Run this to generate a 100 individual by 1000 SNP simulated test set
ped = []
for i, (row, ind) in tqdm.tqdm(enumerate(fam.iloc[:100, :].iterrows())):
    rand_genotype = np.concatenate(
        np.stack(bim.iloc[:1000, :][[4, 5]].apply(np.random.choice, size=2, axis=1))
    )
    ped.append((list(ind.values) + list(rand_genotype)))
ped = np.array(ped)

100it [00:06, 15.61it/s]


In [97]:
ped[:, 5] = np.random.choice([1, 2], size=100)

In [98]:
np.savetxt("MGB_100_1000/MGB_100_1000.ped", ped, delimiter=" ", fmt="%s")

#### BIM

In [99]:
bim[[0, 1, 2, 3]].iloc[:1000, :].to_csv(
    "MGB_100_1000/MGB_100_1000.map", sep="\t", header=None, index=False
)

### 100 x All SNPs

#### PED

In [23]:
fam.values[:10].shape, np.tile(bim[[4, 5]].values.flatten(), (10, 1)).shape, np.concatenate([fam.values[:10], np.tile(bim[[4, 5]].values.flatten(), (10, 1))], axis=1).shape

((10, 6), (10, 3407714), (10, 3407720))

In [27]:
sample_fam = fam.values[:10]

In [31]:
sample_fam[:, 5] = np.random.choice([1, 2], size=10)

In [33]:
np.savetxt("MGB_10_all/MGB_10_all.ped", 
           np.concatenate([sample_fam, np.tile(bim[[4, 5]].values.flatten(), (10, 1))], axis=1),
           fmt="%s")

#### MAP

In [25]:
bim[[0, 1, 2, 3]].to_csv(
    "MGB_10_all/MGB_10_all.map", sep="\t", header=None, index=False
)

### Full set

In [20]:
num = 100
for i in range(int(len(fam)/num)+1):
    start = num*i
    end = num*(i+1)
    if end > len(fam):
        end = len(fam)
    print(start, end)
    np.savetxt("full/MGB_{}_{}.ped".format(start, end), np.concatenate([fam.values[start:end], np.tile(bim[[4, 5]].values.flatten(), (num, 1))], axis=1), fmt="%s")
    if i == 3:
        break

0 100
100 200
200 300
300 400


## Part 1 - Make a SNP summary file

In [3]:
geno = feather.read_feather("../../../cagi6-prs/features/baseline_snps-baseline_plus_samples/ibd_age_old/ibd.age.old.feather")

In [4]:
snps = ["_".join(var.split("_")[:-1]) for var in geno["index"]]
alleles = [var[-1] for var in geno["index"]]
allele_mp = dict(zip(snps, alleles))

In [2]:
snp_map = pd.read_csv("../../../cagi6-prs/snps/ukb.extracted.intersected.mgb.snps.tsv", sep="\t")

In [5]:
subset_snp_map = snp_map[snp_map["UKB_ID"].isin(snps)][["UKB_ID", "MGB_ID"]]

In [6]:
stats = pd.read_csv("../../../cagi6-prs/features/baseline_snps-baseline_plus_samples/ibd_age_old/ibd.age.old.train.stats.tsv", sep="\t")

In [7]:
subset_stats = stats[stats["0"].isin(snps)]

In [8]:
summary_df = pd.merge(subset_snp_map, subset_stats, left_on="UKB_ID", right_on="0")

In [10]:
summary_df["expected_allele"] = summary_df["UKB_ID"].map(allele_mp)

In [14]:
final_df = summary_df[["UKB_ID", "MGB_ID", "expected_allele", "mean", "std"]]

In [15]:
final_df.columns = ["UKB_ID", "MGB_ID", "EXPECTED_ALLELE", "MEAN", "STD"]

In [18]:
final_df.to_csv("MGB_10_all/ibd.age.old.summary.tsv", sep="\t", index=False)

In [25]:
final_df["MGB_ID"].isin(bim[1]).all()

True

In [27]:
final_df["MGB_ID"].to_csv("MGB_10_all/ibd.age.old.extract.txt", header=None, index=False)

## Part 2 - Build a feather file from raw and double check it matches model allele
If alleles don't match, do the 2- thing

In [28]:
raw = pd.read_csv("MGB_10_all/MGB_10_all.raw", delim_whitespace=True)

In [30]:
IIDs = raw["IID"]

In [31]:
raw = raw.iloc[:, 6:].T
raw.columns = IIDs
raw.index.name = "0"

In [33]:
summary = pd.read_csv("MGB_10_all/ibd.age.old.summary.tsv", sep="\t")

In [37]:
snps = ["_".join(var.split("_")[:-1]) for var in raw.index]
alleles = [var[-1] for var in raw.index]
#allele_mp = dict(zip(snps, alleles))

In [46]:
raw_summary = pd.DataFrame({"INDEX":raw.index, "MGB_ID":snps, "ACTUAL_ALLELE":alleles})

In [53]:
merged_summary = pd.merge(raw_summary, summary, on="MGB_ID").set_index("INDEX")

In [58]:
ordered_merged_summary = merged_summary.loc[raw.index]

In [65]:
mismatched_pos = np.where(ordered_merged_summary["ACTUAL_ALLELE"] != ordered_merged_summary["EXPECTED_ALLELE"])[0]

In [70]:
raw.iloc[mismatched_pos, :] = 2 - raw.iloc[mismatched_pos, :]

In [74]:
zraw = raw.subtract(ordered_merged_summary["MEAN"].values, axis="index")
zraw = zraw.div(ordered_merged_summary["STD"].values, axis="index")

In [76]:
feather.read_feather("MGB_10_all/MGB_10_all.zscored.feather")

,0,MGB00001,MGB00002,MGB00003,MGB00004,MGB00005,MGB00006,MGB00007,MGB00008,MGB00009,MGB00010
0,1:1776269_C_A_C,-0.175405,-0.175405,-0.175405,-0.175405,-0.175405,-0.175405,-0.175405,-0.175405,-0.175405,-0.175405
1,1:1781220_T_C_T,-0.584103,-0.584103,-0.584103,-0.584103,-0.584103,-0.584103,-0.584103,-0.584103,-0.584103,-0.584103
2,1:1796616_G_A_G,-0.596522,-0.596522,-0.596522,-0.596522,-0.596522,-0.596522,-0.596522,-0.596522,-0.596522,-0.596522
3,1:1860087_C_T_T,0.138641,0.138641,0.138641,0.138641,0.138641,0.138641,0.138641,0.138641,0.138641,0.138641
4,1:1864526_C_T_T,0.144202,0.144202,0.144202,0.144202,0.144202,0.144202,0.144202,0.144202,0.144202,0.144202
...,...,...,...,...,...,...,...,...,...,...,...
5719,22:39774525_A_G_G,0.572654,0.572654,0.572654,0.572654,0.572654,0.572654,0.572654,0.572654,0.572654,0.572654
5720,22:43561675_T_G_G,0.463974,0.463974,0.463974,0.463974,0.463974,0.463974,0.463974,0.463974,0.463974,0.463974
5721,22:43561982_C_T_T,0.463861,0.463861,0.463861,0.463861,0.463861,0.463861,0.463861,0.463861,0.463861,0.463861
5722,22:43562306_A_G_G,0.474088,0.474088,0.474088,0.474088,0.474088,0.474088,0.474088,0.474088,0.474088,0.474088


#### 100x1000

In [151]:
feather.write_feather(raw.reset_index(), "MGB_100_1000.feather")

In [152]:
feather.read_feather("MGB_100_1000.feather")

,0,MGB00001,MGB00002,MGB00003,MGB00004,MGB00005,MGB00006,MGB00007,MGB00008,MGB00009,...,MGB00091,MGB00092,MGB00093,MGB00094,MGB00095,MGB00096,MGB00097,MGB00098,MGB00099,MGB00100
0,rs3131972_A,1,0,0,2,1,2,1,2,0,...,1,1,1,2,1,0,1,1,0,1
1,1:840327_G_A_A,1,1,0,1,0,0,2,2,1,...,2,1,0,0,1,2,0,2,0,2
2,rs4970382_T,2,1,2,1,0,1,2,1,0,...,1,1,1,2,0,1,1,1,1,1
3,1:846808_C_T_C,1,0,1,0,1,0,0,1,2,...,0,1,0,1,1,1,0,1,0,1
4,Affx-15447216_C,0,1,0,2,2,1,1,0,1,...,1,1,0,2,2,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1:4496256_C_T_T,2,2,0,1,1,1,0,1,1,...,0,2,1,2,1,1,0,1,1,0
996,1:4496659_C_T_T,0,1,0,2,0,2,0,1,1,...,2,0,0,1,1,0,1,1,1,0
997,1:4497097_G_A_A,1,1,1,1,1,1,0,2,2,...,0,1,2,0,2,0,0,1,1,2
998,1:4497118_C_T_C,1,0,2,0,1,1,1,0,1,...,2,1,1,2,0,0,2,1,1,1


In [193]:
stats = pd.DataFrame(data={"0": raw.index[:1000], "mean": np.random.normal(size=1000), "std": np.random.normal(size=1000)})

In [194]:
stats.to_csv("MGB_100_1000.stats.tsv", sep="\t", index=False)

In [198]:
raw = feather.read_feather("MGB_100_1000.feather").set_index("0")

In [226]:
zraw = raw.subtract(stats["mean"].values, axis="index")
zraw = zraw.div(stats["std"].values, axis="index")

In [236]:
feather.write_feather(zraw.reset_index(), "MGB_100_1000.zscored.feather")

In [238]:
zraw

,MGB00001,MGB00002,MGB00003,MGB00004,MGB00005,MGB00006,MGB00007,MGB00008,MGB00009,MGB00010,...,MGB00091,MGB00092,MGB00093,MGB00094,MGB00095,MGB00096,MGB00097,MGB00098,MGB00099,MGB00100
0,,,,,,,,,,,,,,,,,,,,,
rs3131972_A,-0.435274,0.145781,0.145781,-1.016330,-0.435274,-1.016330,-0.435274,-1.016330,0.145781,-1.016330,...,-0.435274,-0.435274,-0.435274,-1.016330,-0.435274,0.145781,-0.435274,-0.435274,0.145781,-0.435274
1:840327_G_A_A,3.971477,3.971477,0.743766,3.971477,0.743766,0.743766,7.199187,7.199187,3.971477,7.199187,...,7.199187,3.971477,0.743766,0.743766,3.971477,7.199187,0.743766,7.199187,0.743766,7.199187
rs4970382_T,-3.526750,-2.746769,-3.526750,-2.746769,-1.966788,-2.746769,-3.526750,-2.746769,-1.966788,-3.526750,...,-2.746769,-2.746769,-2.746769,-3.526750,-1.966788,-2.746769,-2.746769,-2.746769,-2.746769,-2.746769
1:846808_C_T_C,2.722384,1.299216,2.722384,1.299216,2.722384,1.299216,1.299216,2.722384,4.145552,2.722384,...,1.299216,2.722384,1.299216,2.722384,2.722384,2.722384,1.299216,2.722384,1.299216,2.722384
Affx-15447216_C,5.412217,1.196706,5.412217,-3.018806,-3.018806,1.196706,1.196706,5.412217,1.196706,1.196706,...,1.196706,1.196706,5.412217,-3.018806,-3.018806,1.196706,1.196706,1.196706,5.412217,5.412217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1:4496256_C_T_T,-1.126842,-1.126842,0.850135,-0.138353,-0.138353,-0.138353,0.850135,-0.138353,-0.138353,-0.138353,...,0.850135,-1.126842,-0.138353,-1.126842,-0.138353,-0.138353,0.850135,-0.138353,-0.138353,0.850135
1:4496659_C_T_T,-2.201933,-4.277609,-2.201933,-6.353284,-2.201933,-6.353284,-2.201933,-4.277609,-4.277609,-2.201933,...,-6.353284,-2.201933,-2.201933,-4.277609,-4.277609,-2.201933,-4.277609,-4.277609,-4.277609,-2.201933
1:4497097_G_A_A,-0.295714,-0.295714,-0.295714,-0.295714,-0.295714,-0.295714,0.356939,-0.948368,-0.948368,0.356939,...,0.356939,-0.295714,-0.948368,0.356939,-0.948368,0.356939,0.356939,-0.295714,-0.295714,-0.948368


In [237]:
feather.read_feather("../../test.zscored.feather")

,0,MGB00001,MGB00002,MGB00003,MGB00004,MGB00005,MGB00006,MGB00007,MGB00008,MGB00009,...,MGB00091,MGB00092,MGB00093,MGB00094,MGB00095,MGB00096,MGB00097,MGB00098,MGB00099,MGB00100
0,rs3131972_A,-0.435274,0.145781,0.145781,-1.016330,-0.435274,-1.016330,-0.435274,-1.016330,0.145781,...,-0.435274,-0.435274,-0.435274,-1.016330,-0.435274,0.145781,-0.435274,-0.435274,0.145781,-0.435274
1,1:840327_G_A_A,3.971477,3.971477,0.743766,3.971477,0.743766,0.743766,7.199187,7.199187,3.971477,...,7.199187,3.971477,0.743766,0.743766,3.971477,7.199187,0.743766,7.199187,0.743766,7.199187
2,rs4970382_T,-3.526750,-2.746769,-3.526750,-2.746769,-1.966788,-2.746769,-3.526750,-2.746769,-1.966788,...,-2.746769,-2.746769,-2.746769,-3.526750,-1.966788,-2.746769,-2.746769,-2.746769,-2.746769,-2.746769
3,1:846808_C_T_C,2.722384,1.299216,2.722384,1.299216,2.722384,1.299216,1.299216,2.722384,4.145552,...,1.299216,2.722384,1.299216,2.722384,2.722384,2.722384,1.299216,2.722384,1.299216,2.722384
4,Affx-15447216_C,5.412217,1.196706,5.412217,-3.018806,-3.018806,1.196706,1.196706,5.412217,1.196706,...,1.196706,1.196706,5.412217,-3.018806,-3.018806,1.196706,1.196706,1.196706,5.412217,5.412217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1:4496256_C_T_T,-1.126842,-1.126842,0.850135,-0.138353,-0.138353,-0.138353,0.850135,-0.138353,-0.138353,...,0.850135,-1.126842,-0.138353,-1.126842,-0.138353,-0.138353,0.850135,-0.138353,-0.138353,0.850135
996,1:4496659_C_T_T,-2.201933,-4.277609,-2.201933,-6.353284,-2.201933,-6.353284,-2.201933,-4.277609,-4.277609,...,-6.353284,-2.201933,-2.201933,-4.277609,-4.277609,-2.201933,-4.277609,-4.277609,-4.277609,-2.201933
997,1:4497097_G_A_A,-0.295714,-0.295714,-0.295714,-0.295714,-0.295714,-0.295714,0.356939,-0.948368,-0.948368,...,0.356939,-0.295714,-0.948368,0.356939,-0.948368,0.356939,0.356939,-0.295714,-0.295714,-0.948368
998,1:4497118_C_T_C,-11.307576,-20.985076,-1.630075,-20.985076,-11.307576,-11.307576,-11.307576,-20.985076,-11.307576,...,-1.630075,-11.307576,-11.307576,-1.630075,-20.985076,-20.985076,-1.630075,-11.307576,-11.307576,-11.307576


## Part 3 - Phenotype files for dataloading
Build a tsv and an ids file for dataloading

In [130]:
tsv = fam.iloc[:100, :].copy()

In [131]:
tsv.columns = ["FID", "IID", "PAT", "MAT", "SEX", "PHENOTYPE"]

In [132]:
tsv["AGE"] = -9

In [133]:
tsv["FH"] = -9

In [134]:
tsv["ETH"] = "EUR"

In [161]:
tsv.to_csv("MGB_100_1000.tsv", sep="\t")

In [138]:
ids = tsv["IID"].values

In [139]:
np.savetxt("MGB_100_1000.ids.txt", ids, fmt="%s")

## Part 5 - Test dataloading

In [239]:
import sys

In [240]:
sys.path.append("/cellar/users/aklie/cagi6-prs-docker")

In [241]:
import SNPLoader

In [243]:
loader = SNPLoader.get_loader(
    ids_file="../test/MGB_100_1000.ids.txt",
    genotype_file="../test/MGB_100_1000.zscored.feather",
    phenotype_file="../test/MGB_100_1000.tsv",
    disease_column="PHENOTYPE",
    batch_size=100,
    shuffle=True,
    num_workers=2,
)

In [244]:
for batch_num, (snp, pheno, eth, fh) in enumerate(loader):
    print(batch_num)

0


In [245]:
snp.size(), pheno.size(), eth.size(), fh.size()

(torch.Size([100, 1000]),
 torch.Size([100, 1]),
 torch.Size([100, 1]),
 torch.Size([100, 1]))

In [251]:
np.unique(snp[:, 0], return_counts=True)

(array([-1.0163296 , -0.43527448,  0.14578073], dtype=float32),
 array([15, 65, 20]))

# Scratch
Place for old or testing code

# References